<a href="https://colab.research.google.com/github/ziyadamz2/SLM/blob/Dataset/SLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.3/235.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.0/349.0 kB 26.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [9]:
kwargs = dict(split="train", streaming=False)
dataset = load_dataset("OpenLLM-France/Lucie-Training-Dataset", "Wikipedia-fr", **kwargs)

Resolving data files:   0%|          | 0/9491 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/127 [00:00<?, ?it/s]

Wikipedia--fr--003.parquet:   0%|          | 0.00/41.2M [00:00<?, ?B/s]

Wikipedia--fr--014.parquet:   0%|          | 0.00/35.1M [00:00<?, ?B/s]

Wikipedia--fr--015.parquet:   0%|          | 0.00/29.4M [00:00<?, ?B/s]

...

Wikipedia--fr--124.parquet:   0%|          | 0.00/36.9M [00:00<?, ?B/s]

Wikipedia--fr--126.parquet:   0%|          | 0.00/28.2M [00:00<?, ?B/s]

Wikipedia--fr--125.parquet:   0%|          | 0.00/30.6M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
dataset = dataset.to_pandas()

In [11]:
print(dataset.columns)
print(dataset.shape)

Index(['source', 'id', 'language', 'date', 'author', 'url', 'title', 'extra',
       'quality_signals', 'text'],
      dtype='object')
(2647717, 10)


In [12]:
dataset=dataset.drop(columns={'source', 'id', 'language', 'date', 'author', 'url','extra','quality_signals'})

In [13]:
dataset.iloc[3]

,3
title,1953 en Inde
text,"# 1953 en Inde\n\nChronologie de l'Inde\n\n* 1949\n* 1950\n* 1951\n* 1952\n* 1953\n* 1954\n* 1955\n* 1956\n* 1957\n\nCette page concerne les évènements survenus en 1953 en Inde :\n\n## Évènement\n\n* Le gouvernement indien met en place la première commission des classes défavorisées, dirigée par Kaka Kalelkar (en).\n* Schéma modifié de l'enseignement élémentaire (en).\n* 2 mai : Accident du vol BOAC 783 (en) peu après son décollage à Calcutta.\n* 28 septembre : Pacte Nagpur (en), entre les dirigeants politiques indiens. Il conduit à la création de l'État du Maharashtra à partir des zones contiguës de langue marathi de l'État de Bombay, de l'État du Madhya Pradesh et de l'État d'Hyderabad.\n\n## Sortie de film\n\n* Baaz\n* Deux Hectares de terre\n* Devdas\n* Jhansi Ki Rani\n* Ladki\n\n## Littérature\n\n* The Private Life of an Indian Prince (en), roman de Mulk Raj Anand.\n\n## Création\n\n* État d'Andhra (en) : découpe des régions de langue télougou de l'ancienne Présidence de Madras.\n* Indian Airlines\n* Nationalisation d'Air India\n\n## Naissance\n\n* Kambhampati Hari Babu (en), parlementaire.\n* K. Bhagyaraj (en), acteur, réalisateur.\n* Hari Chand, athlète spécialiste des courses de fond.\n* Avinash Balkrishna Patwardhan (en), ingénieur.\n* Suresh Prabhu (en), ministre.\n* Devi Shetty, chirurgien.\n* Ranjit Sinha (en), policier.\n* Varaprasad Rao Velagapalli (en), parlementaire.\n\n## Décès\n\n* Abdullah Yusuf Ali, érudit islamique.\n* Sulaiman Nadvi, écrivain.\n"
